In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from algorithms.MAB import MAB
from algorithms.reward_model import LinearRewardModel, EqualGapRewardModel, CustomRewardModel
from algorithms.bai.fbai import FairBAI
from algorithms.bai.tas import TaS
from algorithms.bai.uniform import UniformBAI
from algorithms.fairness_model import ProportionalFairnessModel, VectorFairnessModel, make_prespecified_fairness_model
from tqdm import tqdm
from numpy.typing import NDArray
import pickle
from scipy.stats import bootstrap
import warnings

plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
plt.rc('text', usetex=True)
SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def compute_confidence(x):
    mean = np.mean(x)
    ci = bootstrap((x,), statistic=np.mean).confidence_interval
    ci = (np.nan_to_num(ci.low, nan =0), np.nan_to_num(ci.high, nan =0))
    return mean, ci

def compute_confidence_violation(x):
    z = x.mean(0)
    mean_ci = bootstrap((x,), statistic=np.mean, axis=0)

    return z, mean_ci.confidence_interval
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()

In [ ]:
K = 10
p0 = 0.9
SOLVER = cp.ECOS
DELTAS = [1e-3, 1e-2, 1e-1]

with open('models/scheduling_env_model/circle_models.pkl', 'rb') as handle:
    THETAs = pickle.load(handle)
    
models = [
    ('Pre-specified model agonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w, p0, agonistic=True), 'Pre-specified model agonistic'),
    ('Pre-specified model antagonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w,  p0, agonistic=False), 'Pre-specified model antagonistic'),
]

In [ ]:
results_prespecified = {
    x: {delta: {} for delta in DELTAS} for x in ['agonistic', 'antagonistic']}
results_thetadep = {
    x: {delta: {} for delta in DELTAS} for x in ['agonistic', 'antagonistic']}
algorithms = []
for delta in DELTAS:
    for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/scheduler/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
            algorithms = results.keys()
        instance = MAB(reward_model=reward_model, fairness_model=None)
        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

        #fig, ax = plt.subplots(1, 1)
        _results = results_prespecified['antagonistic'] if 'antagonistic' in filename else results_prespecified['agonistic']
        for idx, algo_name in enumerate(results.keys()):
            mu_tau, ci_tau = compute_confidence(results[algo_name]['t_vec'])
            mu_perr, ci_err = compute_confidence(results[algo_name]['p_err'])

            w_fair_vec = results[algo_name]['w_vec']

            #fairness_res_fbai = np.array([ np.mean([max(0,np.max(instance.f(instance.THETA) - sample))   for t, sample in enumerate(x)])for x in w_fair_vec])

           #fairness_res_fbai = np.array([max(0,np.max(instance.f(instance.THETA) - sample))  for x in w_fair_vec  for t, sample in enumerate(x)])

            # plt.hist(fairness_res_fbai, bins=100)
            # plt.title(f'{algo_name} - {filename}')
            # plt.show()
            fairness_res_fbai = np.array([max(0,np.max(instance.f(instance.THETA)-x[-1])) for x in w_fair_vec]) 
            mu_viol, ci_viol = compute_confidence_violation(fairness_res_fbai)
            
            print(f'[{algo_name}] {mu_viol} - {ci_viol}')
            #if 'linear' in filename:
            _results[delta][algo_name] = (mu_tau, max(mu_tau - ci_tau[0], ci_tau[1] - mu_tau), mu_viol, np.maximum(mu_viol - ci_viol.low, ci_viol.high - mu_viol) )


In [ ]:
K = 10
p0 = 0.9
DELTA = 1e-2
models_thetadep = [
    ('Proportional fair agonistic_', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=True),'Proportional fair agonistic'),
    ('Proportional fair antagonistic_', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=False),'Proportional fair antagonistic'),
]
results_thetadep = {
    x: {delta: {} for delta in DELTAS} for x in ['agonistic', 'antagonistic']}


for delta in DELTAS:
    for idx, (name, reward_model, fairness_model, filename) in enumerate(models_thetadep):
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/scheduler/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)
        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

        instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
            
        #fig, ax = plt.subplots(1, 1)
        _results = results_thetadep['antagonistic'] if 'antagonistic' in filename else results_thetadep['agonistic']
        for idx, algo_name in enumerate(results.keys()):
            mu_tau, ci_tau = compute_confidence(results[algo_name]['t_vec'])
            mu_perr, ci_err = compute_confidence(results[algo_name]['p_err'])
            #print(f"[{algo_name}] Sample complexity: {mu_tau} ({ci_tau}) - P(err): {mu_perr} ({ci_err})")

            w_fair_vec = results[algo_name]['w_vec']
            fairness_res_fbai = np.array([max(0,np.max(instance.f(instance.THETA)-x[-1])) for x in w_fair_vec]) 
            #fairness_res_fbai = np.array([ np.mean([max(0,np.max(instance.f(instance.THETA) - sample))   for t, sample in enumerate(x)])for x in w_fair_vec])

            mu_viol, ci_viol = compute_confidence_violation(fairness_res_fbai)
        

            #if 'linear' in filename:
            _results[delta][algo_name] = (mu_tau, max(mu_tau - ci_tau[0], ci_tau[1] - mu_tau), mu_viol, np.maximum(mu_viol - ci_viol.low, ci_viol.high - mu_viol) )


In [ ]:
for delta in reversed(DELTAS):
    last_delta = delta == DELTAS[0]
    for idx, algo_name in enumerate(algorithms):
        last_row = idx == len(results.keys()) -1 
        c = []
        m = []

        c_viol = []
        m_viol = []
        for res in [results_prespecified, results_thetadep]:
            for x in ['agonistic','antagonistic']:
                c.append(max(np.nan_to_num(res[x][delta][algo_name][1], nan=0), np.nan_to_num(res[x][delta][algo_name][2], nan=0)))
                m.append(res[x][delta][algo_name][0])
                c_viol.append(np.nan_to_num(res[x][delta][algo_name][3], nan=0))
                m_viol.append(np.nan_to_num(res[x][delta][algo_name][2], nan=0))
                
        #print(results_agonistic[delta][algo_name])
        table_row = "{} {} & {} & ${}$ & ${}$ & ${}$ & ${}$ & ${}$ & ${}$ & ${}$ & ${}$\\\\ {}".format(
            r'\rowcolor[gray]{.95}' if idx % 2 ==0 else '',
            f'$\delta={delta}$' if idx == 0 else '', r'\textsc{' + algo_name + '}', 
            f'{m[0]:.2f} \\pm {c[0]:.2f}',
            f'{m[1]:.2f} \\pm {c[1]:.2f}',
            f'{100*m_viol[0]:.2f}\\% \\pm {100*c_viol[0]:.2f}\\%',
            f'{100*m_viol[1]:.2f}\\% \\pm {100*c_viol[1]:.2f}\\%',
            f'{m[2]:.2f} \\pm {c[2]:.2f}',
            f'{m[3]:.2f} \\pm {c[3]:.2f}',
            f'{100*m_viol[2]:.2f}\\% \\pm {100*c_viol[2]:.2f}\\%',
            f'{100*m_viol[3]:.2f}\\% \\pm {100*c_viol[3]:.2f}\\%',
            '' if not last_row else  '\\midrule' if not last_delta else '\\bottomrule'
        )
        print(table_row)